In [111]:
# Group 16
# Sjoerd Bootsma: 5242053
# Gijs van der Klink: 5389283
# Jelle Weijland: 5093457

#------------------------------------------------------------------------------------------------

import math
import pandas as pd
import os
import numpy as np


data_file_path = 'data/AirportData.xlsx'
data_file_path2 = 'data/Group16.xlsx'

#------------------------------------------------------------------------------------------------

def load_airport_data(file_path, sheet_name='Airport', output_csv_path='AirportData.csv'):
    airport_data = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    airport_data_transposed = airport_data.transpose()
    airport_data_transposed.columns = airport_data_transposed.iloc[0]
    airport_data_transposed = airport_data_transposed[1:]
    
    # Sla het getransponeerde DataFrame op als CSV-bestand
    airport_data_transposed.to_csv(output_csv_path, index=False)    
    return airport_data_transposed

def calculate_distance(lat1, lon1, lat2, lon2):
    """Bereken de afstand in kilometers tussen twee geografische coördinaten."""
    R = 6371  # Aarde straal in kilometers
    lat1, lon1 = math.radians(lat1), math.radians(lon1)
    lat2, lon2 = math.radians(lat2), math.radians(lon2)
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    a = math.sin(delta_lat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(delta_lon / 2)**2
    delta_sigma = 2 * math.asin(math.sqrt(a))
    return R * delta_sigma

def calculate_distance_matrix(latitudes, longitudes):
    """Maak een afstandsmatrix voor de opgegeven breedte- en lengtegraden."""
    num_airports = len(latitudes)
    distance_matrix = [[0 for _ in range(num_airports)] for _ in range(num_airports)]
    for i in range(num_airports):
        for j in range(num_airports):
            if i != j:
                distance_matrix[i][j] = calculate_distance(latitudes[i], longitudes[i], latitudes[j], longitudes[j])
    return distance_matrix

def calculate_distance_dataframe(airport_data):
    """Berekent een DataFrame van afstanden tussen vliegvelden."""
    latitudes = airport_data['Latitude (deg)'].astype(float).values
    longitudes = airport_data['Longitude (deg)'].astype(float).values
    city_names = airport_data['IATA code'].tolist()
    distance_matrix = calculate_distance_matrix(latitudes, longitudes)
    distance_df = pd.DataFrame(distance_matrix, index=city_names, columns=city_names)
    return distance_df.round(1)

def save_and_print_fra_distances(distance_df, output_csv_path='DistanceMatrix_FRA.csv'):
    """Filtert de afstanden vanaf Frankfurt (FRA) en slaat ze op in een CSV-bestand."""
    if 'FRA' in distance_df.index and 'FRA' in distance_df.columns:
        fra_distances = distance_df.loc['FRA']
        filtered_distance_df = pd.DataFrame({"Distance to/from FRA": fra_distances})
        filtered_distance_df.to_csv(output_csv_path, index=True)


def process_demand_data(file_path2, sheet_name):
    import pandas as pd  # Ensure pandas is imported
    
    # Load Excel data and remove the first column
    data = pd.read_excel(
        file_path2, 
        sheet_name=sheet_name, 
        header=None, 
        usecols=lambda x: x != 0  
    )
    data.columns = [f"Column_{i}" for i in range(1, data.shape[1] + 1)]
    data = data.rename(columns={"Column_1": "Departure", "Column_2": "Arrival"})

    # Define column labels for time intervals and days
    time_frames = [
        "00:00-04:00", "04:00-08:00", "08:00-12:00", "12:00-16:00", 
        "16:00-20:00", "20:00-00:00"
    ]
    num_days = 5  # Adjust this based on your dataset; e.g., 5 days
    time_labels = [
        f"Day {day} {time_frame}" 
        for day in range(1, num_days + 1) 
        for time_frame in time_frames
    ]

    # Map columns 3-32 to the generated time labels
    columns_to_label = [f"Column_{i}" for i in range(3, 33)]  # Columns 3 to 32
    label_mapping = dict(zip(columns_to_label, time_labels))
    data = data.rename(columns=label_mapping)

    # Filter rows related to FRA
    fra_related_rows = data[
        (data["Departure"] == "FRA") | (data["Arrival"] == "FRA")
    ].reset_index(drop=True)

    fra_related_rows.to_csv("processed_demand_data.csv", index=False)
    return fra_related_rows


# Call the function
process_demand_data(data_file_path2, 'Group 16')
load_airport_data(data_file_path)
calculate_distance_dataframe(airport_data)
save_and_print_fra_distances(distance_df)



In [107]:
distance_matrix_path = 'DistanceMatrix_FRA.csv'


# Fleet data
fleet_data = pd.DataFrame({
    'Type': ['Small Freighter', 'Mid-size Old Freighter', 'Large Freighter'],
    'Speed': [800, 850, 920],  # Speed in km/h
    'Cargo_Capacity': [23000, 35000, 120000],  # Cargo capacity in kg
    'TAT': [90, 120, 150],  # Average Turn-Around Time in minutes
    'Max_Range': [1500, 3300, 6300],  # Maximum range in km
    'RQ': [1400, 1600, 1800],  # Runway required in meters
    'Lease_Cost': [2143, 4857, 11429],  # Lease cost in EUR/day
    'Fixed_Cost': [750, 1500, 3125],  # Fixed operating cost per flight leg in EUR
    'Time_Cost': [1875, 1938, 3500],  # Cost per hour in EUR
    'Fuel_Cost': [2.5, 5, 9.5],  # Fuel cost parameter
    'Fleet': [2, 2, 1]  # Fleet count
    })


def calculate_operating_costs(distances, fleet_data):
    total_costs_df = pd.DataFrame(index=distances.index)

    for _, row in fleet_data.iterrows():
        type_name = row['Type']
        fixed_cost = row['Fixed_Cost']
        time_cost_param = row['Time_Cost']
        fuel_cost_param = row['Fuel_Cost']
        airspeed = row['Speed']

        # Fixed operating cost (C^k_X)
        fixed_operating_cost = fixed_cost

        # Time-based costs (C^k_Tij)
        time_based_costs = (time_cost_param * distances / airspeed).round(2)

        # Fuel costs (C^k_Fij)
        fuel_costs = (fuel_cost_param * 1.42 *0.97*  distances / 1.5).round(2)

        # Total operating cost (C^k_ij)
        total_costs = fixed_operating_cost + time_based_costs + fuel_costs
        total_costs_df[type_name] = total_costs.round(2)

    return total_costs_df

def calculate_and_save_flight_times(fra_distances, fleet_data, output_csv_path):

    # Maak een DataFrame om vluchtduur op te slaan
    flight_times_df = pd.DataFrame(index=fra_distances.index)

    # Bereken vluchtduur voor elk vliegtuigtype en voeg het toe aan de DataFrame
    for _, row in fleet_data.iterrows():
        type_name = row['Type']
        speed = row['Speed']
        tat = row['TAT']
        
        # Bereken vluchtduur in minuten en rond af naar het dichtstbijzijnde veelvoud van 6
        flight_times_df[type_name] = (
            ((((fra_distances / speed) + 0.5) * 60) + tat)
            .apply(lambda x: int(np.ceil(x / 6.0) * 6))  # Ronden binnen de berekening
        )

    # Sla de vluchtduur op in een CSV-bestand
    flight_times_df.to_csv(output_csv_path, index=True)
    


distances = pd.read_csv(distance_matrix_path, index_col=0)["Distance to/from FRA"]
operating_costs = calculate_operating_costs(distances, fleet_data)
# output_csv_path = 'OperatingCosts_FRA.csv'
operating_costs.to_csv(output_csv_path, index=True)


calculate_and_save_flight_times(distances, fleet_data, 'FlightTimes_FRA.csv')

In [108]:
#Initialize dynamic model

class Node():
    # Initializes a Node with time, airport, and default values for links, profit, and time elapsed
    def __init__(self, Time, airport):
        self.Time = Time
        self.Airport = airport
        self.inLink = None
        self.outLink = None
        self.Tot_profit = None
        self.Time_elapsed = time(0,0)

    # Adds an incoming link to the node
    def add_inLink(self, link):
        self.inLink = link

    # Adds an outgoing link to the node
    def add_outLink(self, link):
        self.outLink = link

    # Calculates total profit at this node
    def add_Profit(self):
        self.Tot_profit = self.outLink.Profit + self.outLink.To.Tot_profit

    # Updates the time elapsed for this node
    def add_Time(self, onground=False):
        if onground is True:
          self.Time_elapsed = add_time(self.outLink.To.Time_elapsed, 0, 6)
        else:
          self.Time_elapsed = add_time(self.outLink.Flighttime, self.outLink.To.Time_elapsed.hour, self.outLink.To.Time_elapsed.minute)

    # String representation of the node
    def __str__(self):
        return f'Node at time: {self.Time} and airport: {self.Airport}'

    # Representation used for debugging
    def __repr__(self):
        return f'{self.Time}_{self.Airport}_{round(self.Tot_profit)}'

class Link():
    # Initializes a Link with origin, destination, distance, cost, revenue, profit, cargoload , and flighttime
    def __init__(self, origin, destination, distance, cost, revenue, profit, cargoload, flighttime):
        self.From = origin
        self.To = destination
        self.Distance = distance
        self.cost = cost
        self.revenue = revenue
        self.Profit = profit
        self.cargoload = cargoload
        self.Flighttime = flighttime

In [109]:
df_operatingcost = pd.read_csv("OperatingCosts_FRA.csv", index_col=0)
df_flight_times = pd.read_csv("FlightTimes_FRA.csv", index_col=0)
df_distance = pd.read_csv("DistanceMatrix_FRA.csv", index_col=0)
df_airportdata = pd.read_csv("AirportData.csv", index_col=0)

def calc_cost(k, i, j):
    # Controleer of één van de luchthavens Frankfurt is
    if i == 'FRA':
        # Kosten voor vertrek vanuit Frankfurt naar luchthaven j
        cost = df_operatingcost.loc[j, k]
    elif j == 'FRA':
        # Kosten voor aankomst in Frankfurt vanuit luchthaven i
        cost = df_operatingcost.loc[i, k]   
    else:
        raise ValueError("Eén van de luchthavens moet Frankfurt zijn")

    return cost

def calc_flight_time(k, i, j):
    # Controleer of één van de luchthavens Frankfurt is
    if i == 'FRA':
        # Vluchttijd voor vertrek vanuit Frankfurt naar luchthaven j
        flight_time = df_flight_times.loc[j, k]
    elif j == 'FRA':
        # Vluchttijd voor aankomst in Frankfurt vanuit luchthaven i
        flight_time = df_flight_times.loc[i, k]
    else:
        raise ValueError("Eén van de luchthavens moet Frankfurt zijn")

    return flight_time

def calc_distance(i, j):
    # Controleer of één van de luchthavens Frankfurt is
    if i == 'FRA':
        # Afstand vanuit Frankfurt naar luchthaven j
        distance = df_distance.loc[j, 'Distance to/from FRA']
    elif j == 'FRA':
        # Afstand naar Frankfurt vanuit luchthaven i
        distance = df_distance.loc[i, 'Distance to/from FRA']
    else:
        raise ValueError("Eén van de luchthavens moet Frankfurt zijn")
    
    return distance

def calc_revenue(distance, flow_i_j):
    revenue = 0.26 * distance * flow_i_j
    return revenue

def check_flightfeasability(aircraft_type, origin, destination, distance, fleet_data):
    check = True

    # Haal maximale vliegbereik en benodigde baanlengte op voor het vliegtuigtype
    max_range = fleet_data.loc[fleet_data['Type'] == aircraft_type, 'Max_Range'].values[0]
    min_runway_required = fleet_data.loc[fleet_data['Type'] == aircraft_type, 'RQ'].values[0]

    # Controleer of de afstand binnen het bereik van het vliegtuig ligt
    if max_range < distance:
        check = False

    # Haal de lengte van de startbaan op voor vertrek- en aankomstluchthaven uit de CSV
    runway_origin = df_airportdata.loc[df_airportdata['IATA code'] == origin, 'Runway (m)'].values[0]
    runway_destination = df_airportdata.loc[df_airportdata['IATA code'] == destination, 'Runway (m)'].values[0]

    # Controleer of de startbanen lang genoeg zijn
    if min_runway_required > runway_origin or min_runway_required > runway_destination:
        check = False

    return check

  
distance = calc_distance('FRA', 'LHR')
cost = calc_cost('Small Freighter', 'FRA', 'LHR')
flight_time = calc_flight_time('Small Freighter', 'FRA', 'LHR')
revenue = calc_revenue(distance, 100)
print(f"Kosten: {cost}, Vluchttijd: {flight_time} minuten, Inkomsten: {revenue}")

check = check_flightfeasability("Small Freighter", "FRA", "LHR", distance, fleet_data)
print(f"Is de vlucht mogelijk? {'Ja' if check else 'Nee'}")


Kosten: 3787.89, Vluchttijd: 174 minuten, Inkomsten: 17024.8
Is de vlucht mogelijk? Ja
